https://chatgpt.com/share/687d132f-70f0-8003-8800-16c45376f179

i extract subtitle from youtube video
the video is 2hours long
i want summarize with Key points of the video using llm model
so i chunk subtitle every 10 minutes and input each chunk to model to summarize
eg. 1 time summarize for 1 chunk
and then I combine all the sumarize together
I use this method because llm model seem to cunfuse when I input the large amount of data into it
so I chunk it first and do all above
is it a good method or any method suggest that better than this? 

to do
- try chunking in token 
    - 1,000–2,000 tokens per chunk is the safe sweet spot
    - Add 10–20% overlap to catch mid-topic cuts

In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.youtube.com/watch?v=ZCoK9nakeJE"

response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
title = soup.title.string.replace(" - YouTube", "").strip()

In [2]:
import re

# get video title name
def clean_title(text):
    return re.sub(r'[^0-9a-zA-Z\u0E00-\u0E7F\.]', '', text)
title = clean_title(text=title)

# get video id
match = re.search(r"(?:v=|\/)([0-9A-Za-z_-]{11}).*", url)
if match:
    video_id = match.group(1)

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi

# Fetch transcript (auto-captions or uploaded)
transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['th', 'en'])

# Optionally, save to file
file_name = f'{title}_{video_id}'
with open(f"{file_name}_subtitle.txt", "w", encoding="utf-8") as f:
    for entry in transcript:
        f.write(f"{entry['start']:.2f}s: {entry['text']}\n")
        # f.write(f"{entry['text']}\n")

In [ ]:
import re

def load_subtitles(file_path):
    subtitles = []
    with open(file_path, encoding='utf-8') as f:
        for line in f:
            match = re.match(r'([0-9.]+)s:\s(.+)', line.strip())
            if match:
                start_time = float(match.group(1))
                text = match.group(2)
                subtitles.append({'start': start_time, 'text': text})
    return subtitles

def chunk_subtitles(subtitles, chunk_size=60, overlap=20):
    """
    Chunk subtitles into segments of `chunk_size` seconds with `overlap` seconds.
    """
    chunks = []
    max_time = subtitles[-1]['start']
    start_time = 0

    while start_time <= max_time:
        end_time = start_time + chunk_size
        chunk_text = []
        for entry in subtitles:
            if start_time <= entry['start'] < end_time:
                chunk_text.append(entry['text'])
        if chunk_text:
            chunks.append({
                'start': start_time,
                'end': end_time,
                'text': ' '.join(chunk_text)
            })
        start_time += chunk_size - overlap
    return chunks

subtitles = load_subtitles(f"{file_name}_subtitle.txt")
chunk_dict = chunk_subtitles(subtitles, chunk_size=600, overlap=60)
# chunk_dict = chunk_subtitles(subtitles, chunk_size=300, overlap=30)

In [30]:
# import re

# def read_text_file(file_path):
#     """Read text from a file and return as a string."""
#     with open(file_path, 'r', encoding='utf-8') as f:
#         return f.read()

# def remove_timestamps(text):
#     """
#     Remove timestamps at the start of each line, e.g., '0.12s: '
#     """
#     # Remove pattern: start of line, optional whitespace, numbers, dot, numbers, 's:', optional whitespace
#     cleaned_text = re.sub(r'^\s*\d+\.\d+s:\s*', '', text, flags=re.MULTILINE)
#     return cleaned_text

# def chunk_text(text, max_tokens, overlap):
#     """
#     Chunk text into segments with a max token limit and overlap.
#     """
#     words = text.split()
#     chunks = []
#     start = 0

#     while start < len(words):
#         end = min(start + max_tokens, len(words))
#         chunk = ' '.join(words[start:end])
#         chunks.append(chunk)
#         start += max_tokens - overlap  # slide window with overlap

#     return chunks

# # Example usage:
# file_path = f"{file_name}_subtitle.txt"
# text = read_text_file(file_path)
# cleaned_text = remove_timestamps(text)

# chunks = chunk_text(cleaned_text, max_tokens=200, overlap=60)

# for i, chunk in enumerate(chunks):
#     print(f"Chunk {i+1}:\n{chunk}\n{'-'*50}")

In [12]:
chunk_dict[0]

{'start': 0,
 'end': 600,
 'text': 'หุ้น DCC ขอมาก็จัดให้สวัสดีครับผมผิหวัง นะครับคุณกัลฟังพcสนี้เป็นล้านเลยเหรอพี่  Now นะครับอัปเดตทุกสถานการณ์การลงทุน ที่ส่งผลกับเงินเบาของทุกคนนะครับผมคลิป นี้ตามสั่งครับตามรีquestเลยครับคือใน ช่องผมนี่ก็จะมีทำเรื่องเกี่ยวกับหุ้น ประมาณว่าแบบเป็นหุ้น Value หุ้นที่มี ราคาค่อนข้างถูกถ้าเทียบกับ PE หรือว่า ค่าต่างๆนะครับหรือแม้กระทั่งหุ้นปันผล นอกเหนือจากหุ้นโกดหุ้นเติบโตหุ้นที่เรา มักจะได้ยินชื่อหุ้นหุ้นที่มีกระแสหรือ ว่าหุ้นที่คิดว่านักลงทุนเพื่อนๆจะได้ ความรู้ครับก็จะมีหุ้นประเภท Value เหมือนกันแล้วจริงๆอ่ะชื่อหุ้นเนี้ยมันมา เป็นพักๆครับมันไม่ได้มาบ่อยก็คือแบบผ่าน ไปสักพักก็จะโผล่มาคอมเมนต์นึงขอถึงหุ้น ตัวนี้ครับแล้วช่วงนี้ก็เป็นช่วงที่ผมก็ ไล่ไล่ทำหุ้นตามที่รีquวสเอาไว้ครับเพราะ ว่าผมก็จะลิสไว้ใน Excel อยู่แล้วว่าใคร ขออะไรบ้างครับแล้วก็ไล่ๆทำนะครับดังนั้น เนี่ยผมก็เลยบอกก่อนว่าถ้าหุ้นตัวไหน เนี่ยถ้าผมเนี่ยสามารถเข้าใจได้หรือว่า ติดตามหุ้นตัวนี้แล้วก็เอามาอธิบายได้โดย ที่มั่นใจว่ามันจะไม่ผิดนะครับแต่ว่าข้อ มูลเนี่ยอาจจะไม่ตรงซะทีเดียวกับหลายๆ สื่อน

In [13]:
# # subtitle = [subtitle['text'] for subtitle in subtitles]
# with open(f"{file_name}_subtitle.txt", 'r') as f:
#     contents = f.read()

In [14]:
title = file_name.split('_')[0]

## Use “Key points extraction” instead of general summary
## Use hierarchical summarization

In [15]:
# === PROMPTS ===
KEY_POINTS_PROMPT = """
You are a helpful assistant.
Below is a section of a video transcript.
Extract the key points using clear bullet points.
Label each point with one of these: [Fact], [Idea], [Quote], [Question], [Action].
Be precise. Keep points short and factual.
Do not add commentary or extra text.

Transcript:
\"\"\"
{chunk_text}
\"\"\"
"""

SECTION_MERGE_PROMPT = """
Below are key points extracted from multiple chunks of a section.
Merge them into a single list:
- Remove duplicates or near-duplicates.
- Group similar ideas if needed.
- Keep clear bullet points.
- Preserve labels like [Fact], [Idea], etc.

Key Points:
\"\"\"
{key_points_text}
\"\"\"
"""

FINAL_SUMMARY_PROMPT = """
Below are the section-level key points for the entire video.
Write a single, well-organized summary:
- Merge and deduplicate.
- Organize by themes.
- Keep [Fact], [Idea], [Quote], [Action], etc.
- Present clearly for a slide deck or notes.
- Do not add fluff.

All Section Points:
\"\"\"
{all_sections_text}
\"\"\"
"""

# === HELPERS ===

import ollama
def call_llm(prompt: str):
    response = ollama.chat(
        model="llama3",
        messages=[
            {"role": "system", "content": "You are a precise simplify assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    return response["message"]["content"]

In [16]:
# === PIPELINE ===

chunk_key_points = []
print("Extracting key points for each chunk...")
for i, chunk in enumerate(chunk_dict):
    prompt = KEY_POINTS_PROMPT.format(chunk_text=chunk['text'])
    result = call_llm(prompt)
    print(f"Chunk {i+1} done.")
    chunk_key_points.append(result)

# Group into sections, e.g., 3 chunks per section
SECTION_SIZE = 3
section_key_points = []

print("\nMerging chunks into sections...")
for i in range(0, len(chunk_key_points), SECTION_SIZE):
    group = chunk_key_points[i:i+SECTION_SIZE]
    group_text = "\n\n".join(group)
    prompt = SECTION_MERGE_PROMPT.format(key_points_text=group_text)
    result = call_llm(prompt)
    print(f"Section {i//SECTION_SIZE + 1} done.")
    section_key_points.append(result)

# Final merge
print("\nMerging all sections into final summary...")
all_sections_text = "\n\n".join(section_key_points)
final_prompt = FINAL_SUMMARY_PROMPT.format(all_sections_text=all_sections_text)
final_summary = call_llm(final_prompt)

print("\n✅ Final Summary:\n")
print(final_summary)

Extracting key points for each chunk...
Chunk 1 done.
Chunk 2 done.
Chunk 3 done.

Merging chunks into sections...
Section 1 done.

Merging all sections into final summary...

✅ Final Summary:

Here is a single, well-organized summary:

**Company Overview:**

* Ceramic tile manufacturer with strong presence in Thailand
* 96% domestic sales, with declining revenue (8,400 -> 7,700 -> 7,000) and net profit (1,700 -> 1,600 -> 1,100)

**Financial Performance:**

* Revenue decreased YoY since 2020 due to decline in demand
* Net profit declined YoY since 2020 due to revenue decrease and increasing average price per unit

**Industry Trends:**

* Decline in ceramic tile industry demand in Thailand, attributed to consumer behavior changes, competition increase, and economic uncertainties

**Management's Outlook:**

* Focusing on cost reduction, operational efficiency improvement, and product diversification to drive growth
* Gradual recovery expected over the long term

**Share Price Performance

In [17]:
with open(f"{file_name}_summary_v2.txt", "w", encoding="utf-8") as f:
    f.write(final_summary)

## Try Map-Reduce with embeddings (advanced)